In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

## q1 ... finished

In [ ]:

# check if you have not missing value
home_team = pd.read_csv("tennis_data/home_team.csv")
away_team = pd.read_csv("tennis_data/away_team.csv")
home_team = home_team[['full_name','player_id']]
away_team = away_team[['full_name','player_id']]
all_teams = pd.concat([home_team , away_team] ,join = 'outer')
all_teams.drop_duplicates(subset=['player_id'] , inplace = True)
all_teams.sort_values('full_name' , inplace = True)
all_teams.reset_index(drop=True , inplace = True)
num_players = all_teams['player_id'].nunique()
print(f'the number of players is: {num_players}')
all_teams


## q3 ... finished 

In [ ]:
home_team = pd.read_csv("tennis_data/home_team.csv")
away_team = pd.read_csv("tennis_data/away_team.csv")
event = pd.read_csv("tennis_data/event.csv")

home_team = home_team[['match_id','player_id' ,'full_name']]
away_team = away_team[['match_id','player_id' ,'full_name']]

all_home = home_team.drop_duplicates(subset = ['match_id'])
all_away = away_team.drop_duplicates(subset = ['match_id'])
all_play = pd.concat([all_home , all_away])
all_play = all_play.groupby('full_name').agg(
    num_game = ('player_id' , 'count')
)

players = pd.merge(home_team , away_team , how = "inner" , on = "match_id" , suffixes=["_home" , "_away"])
players.drop_duplicates(subset = ['match_id'] , inplace = True)


event = event[event['winner_code'].notnull()].drop_duplicates(subset=['match_id']).sort_values('match_id')
event = event[['match_id' , 'winner_code']]
event.reset_index(drop=True , inplace = True)
    
final = pd.merge(players , event , on = 'match_id' , how = 'inner')
final_home = final[final['winner_code'] == 1.0].copy()
final_away = final[final['winner_code'] == 2.0].copy()
final_home = final_home[['full_name_home' , 'player_id_home']].sort_values('full_name_home').rename(columns={'player_id_home':'player_id' , 'full_name_home' :'full_name'})
final_away = final_away[['full_name_away' , 'player_id_away']].sort_values('full_name_away').rename(columns={'player_id_away':'player_id' , 'full_name_away' :'full_name'})

final = pd.concat([final_home , final_away])
final.sort_values('full_name' , inplace = True)
final = final.groupby('full_name').agg(
    num_won = ('player_id' , 'count')
)

final = pd.merge(final , all_play , on = 'full_name' , how = 'left')
final['win_percentage'] = final['num_won'].div(final['num_game']) * 100
final.sort_values('win_percentage' , ascending=False)
# ax = sns.barplot(num_win[:4], x="full_name", y="count")
# you can use number of games and win_games

,num_won,num_game,win_percentage
full_name,,,
"Oluwadare, Hephzibah",1,1,100.000000
"Bu, Yunchaokete",1,1,100.000000
"Wu, Yibing",2,2,100.000000
"Marti, Yann",2,2,100.000000
"Sinner, Jannik",15,16,93.750000
...,...,...,...
"Alkotop, Mousa",1,15,6.666667
"De Dios, Felipe",1,15,6.666667
"Ilkel, Berk",1,16,6.250000


## q5 ... finished

In [ ]:
# import data and clear
time = pd.read_csv("tennis_data/time.csv")
time.drop_duplicates(subset = ['match_id'] , inplace=True)
time.dropna(subset = ['period_1'] , inplace = True)
time.dropna(subset = ['period_2'] , inplace = True)

# check if you have period_4 or period_5
print(time['period_4'].notnull().any())
print(time['period_5'].notnull().any())

# drop period_4 and period_5 and fill null with 0
time.drop(['period_4' , 'period_5' ,'current_period_start_timestamp']  , axis = 'columns', inplace = True )
values = {'period_3':0}
time.fillna(value = values , inplace = True)
time.loc[time['period_3'] == 0 , 'num_set'] = 2
time.loc[time['period_3'] != 0 , 'num_set'] = 3
finall = time.groupby('num_set').agg(
    number_of_sets = ('num_set' , 'count')
)
total = finall['number_of_sets'].sum()
finall['number_of_set_percentage'] = (finall['number_of_sets'].div(total)) * 100
finall

## q7 ... finished

In [ ]:
# import data
aces = pd.read_csv("tennis_data/period.csv")
aces = aces[aces['period'] == 'ALL'][aces['statistic_name'] == 'aces'].drop(['period' , 'statistic_category_name' , 'compare_code' ,'statistic_type' , 'value_type' , 'home_value' , 'away_value' , 'home_total' , 'away_total'] , axis='columns')
aces = aces.drop_duplicates(subset=['match_id']).sort_values('match_id')
aces = aces.astype({'home_stat':'int32' , 'away_stat':'int32'})

# check home&away stat is not null
aces.dropna(subset = ['home_stat' , 'away_stat'] , inplace=True)
aces['number_of_aces'] = aces['home_stat'].add(aces['away_stat'])
aces = aces.drop(['statistic_name' , 'home_stat' ,  'away_stat'] , axis = 'columns').sort_values('number_of_aces' , ascending=False)
aces.reset_index(drop=True , inplace = True)
mean = aces['number_of_aces'].describe()
mean


## q9 ... finished

In [ ]:
from datetime import datetime

# import data
time = pd.read_csv("tennis_data/time.csv")
home_team = pd.read_csv("tennis_data/home_team.csv")
away_team = pd.read_csv("tennis_data/away_team.csv")
event = pd.read_csv("tennis_data/event.csv")
tournament = pd.read_csv("tennis_data/tournament.csv")

# clear data & choose desired data & seprate time of matches
date = time.drop_duplicates(subset = 'match_id').dropna(subset = 'current_period_start_timestamp').copy()
date = date[['match_id' , 'current_period_start_timestamp']]
date = date.astype({'current_period_start_timestamp':'int32'})
date['play_in_month'] = 0
for dt in date['match_id']:
    date.loc[date['match_id'] == dt , 'play_in_month']  = datetime.fromtimestamp(int(date[date['match_id'] == dt]['current_period_start_timestamp'])).month


month_1 = date[date['play_in_month'] == 1]
month_2 = date[date['play_in_month'] == 2]
month_3 = date[date['play_in_month'] == 3]
month_4 = date[date['play_in_month'] == 4]


home_team = home_team[['full_name' , 'player_id' , 'match_id']]
away_team = away_team[['full_name' , 'player_id' , 'match_id']]
players = pd.merge(home_team , away_team , how = "inner" , on = "match_id" , suffixes=["_home" , "_away"])
players.drop_duplicates(subset = ['match_id'] , inplace = True)


event = event[event['winner_code'].notnull()].drop_duplicates(subset=['match_id']).sort_values('match_id')
event = event[['match_id' , 'winner_code']]
event.reset_index(drop = True , inplace = True)

winner = pd.merge(players , event , on = 'match_id' , how = 'inner')

winner.loc[winner['winner_code'] == 1 , 'winner_name'] = winner['full_name_home']
winner.loc[winner['winner_code'] == 1 , 'winner_id'] = winner['player_id_home']
winner.loc[winner['winner_code'] == 2 , 'winner_name'] = winner['full_name_away']
winner.loc[winner['winner_code'] == 2 , 'winner_id'] = winner['player_id_away']
winner = winner[['match_id' , 'winner_name' , 'winner_id']]


tournament = tournament[['match_id' , 'tournament_id']]
tournament.drop_duplicates(subset = 'match_id' , inplace = True)

play_tour = pd.merge(winner , tournament , on = 'match_id' , how = 'inner')
month_1 = pd.merge(month_1 , play_tour , on = 'match_id' , how = 'inner')
month_2 = pd.merge(month_2 , play_tour , on = 'match_id' , how = 'inner')
month_3 = pd.merge(month_3 , play_tour , on = 'match_id' , how = 'inner')
month_4 = pd.merge(month_4 , play_tour , on = 'match_id' , how = 'inner')
month_1 = month_1[['winner_name' , 'winner_id' , 'tournament_id']]
month_2 = month_2[['winner_name' , 'winner_id' , 'tournament_id']]
month_3 = month_3[['winner_name' , 'winner_id' , 'tournament_id']]
month_4 = month_4[['winner_name' , 'winner_id' , 'tournament_id']]


monthes = [month_1 , month_2 , month_3 , month_4]
finall = pd.DataFrame({'tournament_id':1234 , 'month':[1,2,3,4] , 'winner_name':'A' , 'number_of_win':0}) # generate finall dataframe with default values
for indx , mnth in enumerate(monthes):
    counts = mnth.groupby(['tournament_id' , 'winner_name']).size().reset_index(name = 'count')
    max_counts = counts.groupby('tournament_id')['count'].transform('max')
    result = counts[counts['count'] == max_counts].sort_values(['count' , 'tournament_id'] , ascending=False)
    result = result.astype({'winner_name':'str'})
    finall.loc[finall['month'] == indx + 1 , 'winner_name'] = result.iloc[0]['winner_name']
    finall.loc[finall['month'] == indx + 1 , 'number_of_win'] = result.iloc[0]['count']
    finall.loc[finall['month'] == indx + 1 , 'tournament_id'] = result.iloc[0]['tournament_id']

finall



## q11 ... finished 

In [ ]:
# import data and clear
time = pd.read_csv("tennis_data/time.csv")
time.sort_values('match_id' , inplace = True)
time.drop_duplicates(subset = ['match_id'] , inplace=True)
time.dropna(subset = ['period_1'] , inplace = True)
time.dropna(subset = ['period_2'] , inplace = True)

# check if you have period_4 or period_5
print(time['period_4'].notnull().any())
print(time['period_5'].notnull().any())

# drop period_4 and period_5 and fill null with 0
time.drop(['period_4' , 'period_5' ,'current_period_start_timestamp']  , axis = 'columns', inplace = True )
values = {'period_3':'0'}
time.fillna(value = values , inplace = True)
# casting type to int32
time = time.astype({'period_1':'int32' , 'period_2':'int32' , 'period_3':'int32'})

# calculate match_time
time['period1 + period2'] = time['period_1'].add(time['period_2'])
time['total_period'] = time['period1 + period2'].add(time['period_3'])
mean_duration = time['total_period'].mean()
mean_duration

# calculate time
def convert(x):
    hour = x // 3600
    minute = (x - (3600*hour)) // 60
    seconds = x - ((3600*hour) + (minute*60))
   
    return "%d:%02d:%02d" % (hour, minute, seconds)

print(convert(mean_duration))

## q13 ... finished 

In [ ]:
# import data 
home_team = pd.read_csv("tennis_data/home_team.csv")
away_team = pd.read_csv("tennis_data/away_team.csv")

# choose desired columns & drop duplicate data & groupby based on country
home_team = home_team[['full_name' , 'player_id' , 'plays']]
away_team = away_team[['full_name' , 'player_id' , 'plays']]
home_team.dropna(subset=['plays'] , inplace = True)
away_team.dropna(subset= ['plays'] , inplace = True)
data = pd.concat([home_team , away_team])
data.drop_duplicates(subset = ['player_id'] , inplace = True)

# generate desired info
finall = data.groupby('plays').agg(
    number_of_players = ('plays' , 'count')
)
total = finall['number_of_players'].sum()
finall['number_of_players_percentage'] = (finall['number_of_players'].div(total)) * 100
finall

## q15 ... finished 

In [ ]:
# import data 
home_team = pd.read_csv("tennis_data/home_team.csv")
away_team = pd.read_csv("tennis_data/away_team.csv")
venue = pd.read_csv("tennis_data/venue.csv")

## choose desired columns & drop duplicate data & groupby based on country
# 1. home_team & away_team 
home_team = home_team[['country' , 'player_id']]
away_team = away_team[['country' , 'player_id']]

home_team.dropna(subset=['country'] , inplace = True)
away_team.dropna(subset= ['country'] , inplace = True)

# 2. total_player
total_player = pd.concat([home_team , away_team])
total_player.drop_duplicates(subset = ['player_id'] , inplace = True)
total_player = total_player.groupby('country').agg(
    total_player = ('player_id' , 'count')
)

home_team.drop_duplicates(subset = ['player_id'] , inplace = True)
away_team.drop_duplicates(subset = ['player_id'] , inplace = True)

home_team = home_team.groupby('country').agg(
    number_of_home_players = ('player_id' , 'count')
)
away_team = away_team.groupby('country').agg(
    number_of_away_players = ('player_id' , 'count')
)

# 3. venue  
venue.drop_duplicates(subset = 'match_id' , inplace = True)
venue = venue[['country' , 'match_id']].sort_values('country')
venue = venue.groupby('country').agg(
    number_of_game_played = ('match_id','count')
)

# merging data
total = pd.merge(home_team , away_team , on = 'country' , how = 'outer').fillna(0)
total = pd.merge(total , total_player , on = 'country' , how = 'outer')
total = pd.merge(total , venue  , on = 'country' , how = 'outer').fillna(0)

# generate desired info
all_countries = total.shape[0]

have_player = (total[total['total_player'] !=  0].shape[0]) / all_countries * 100

match_organizer = (total[total['number_of_game_played'] !=  0].shape[0]) / all_countries * 100

have_player_match_organizer = (total[total['number_of_game_played'] !=  0][total['total_player'] !=  0].shape[0]) / all_countries * 100

just_organizer = (total[total['number_of_game_played'] !=  0][total['total_player'] ==  0].shape[0]) / all_countries * 100

just_player = (total[total['number_of_game_played'] ==  0][total['total_player'] !=  0].shape[0]) / all_countries * 100

# data = [have_player_match_organizer, just_organizer, just_player]
# # Creating plot
# fig = plt.figure(figsize=(7, 7))
# plt.pie(data , labels = ['have_player_match_organizer' , 'just_organizer' , 'just_player'])
# plt.show()

## q17 ... finished

In [ ]:
time = pd.read_csv("tennis_data/period.csv")
time = time[time['period'] == 'ALL'][time['statistic_name'].isin(['service_games_played' , 'service_games_won'])].drop_duplicates(['match_id' , 'statistic_name']).sort_values(['match_id'])
time = time[['match_id' , 'statistic_name' , 'home_stat' , 'away_stat']]
time = time.astype({'home_stat':'int32' , 'away_stat':'int32'})
time['total'] = time['home_stat'].add(time['away_stat'])
final = time.pivot_table(values = 'total' , columns='statistic_name' , index = 'match_id')
out = final[final['service_games_played']<final['service_games_won']].index
final.drop(out , inplace = True)
final['breake_serve'] = final['service_games_played'].subtract(final['service_games_won'])
final.sort_values('breake_serve' , ascending = False)
# sns.boxplot(data=final['breake_serve'])
# sns.violinplot(x=final['breake_serve'])
# sns.displot(final, x="breake_serve", binwidth=3)
g = sns.catplot(
    data=final,
    x="breake_serve",
    kind="box", orient="h",
     margin_titles=True,
    height=2.0, aspect=5,
)
g.set(xlabel="Breake_serve", ylabel="count")
final['breake_serve'].describe()